In [1]:
from sympy import *
import math
import numpy as np

### 8.1

In [2]:
A=0.25*0.25/2

In [3]:
def ElementStiffness(bi, bj, bk, ci, cj, ck, A):
    kd_1=1/(4*A)*Matrix([[bi**2,bi*bj,bi*bk],
                      [bi*bj,bj**2,bj*bk],
                      [bi*bk,bj*bk,bk**2]])
    kd_2=1/(4*A)*Matrix([[ci**2,ci*cj,ci*ck],
                      [ci*cj,cj**2,cj*ck],
                      [ci*ck,cj*ck,ck**2]])
    return kd_1 + kd_2

In [4]:
bi=-0.25
bj=0.25
bk=0
ci=0
cj=-0.25
ck=0.25

In [5]:
kd1=ElementStiffness(bi, bj, bk, ci, cj, ck, A)

In [6]:
kd3=kd4=kd1

In [7]:
bi=0.25
bj=-0.25
bk=0
ci=0
cj=0.25
ck=-0.25

In [8]:
kd2=ElementStiffness(bi, bj, bk, ci, cj, ck, A)

In [9]:
kd1

Matrix([
[ 0.5, -0.5,    0],
[-0.5,  1.0, -0.5],
[   0, -0.5,  0.5]])

In [10]:
kd2

Matrix([
[ 0.5, -0.5,    0],
[-0.5,  1.0, -0.5],
[   0, -0.5,  0.5]])

In [11]:
inp = [[-0.25, 0.25, 0, 0, -0.25, 0.25, A, 0, 1, 3],
        [0.25, -0.25, 0, 0, 0.25, -0.25, A, 4, 3, 1],
        [-0.25, 0.25, 0, 0, -0.25, 0.25, A, 1, 2, 4],
        [-0.25, 0.25, 0, 0, -0.25, 0.25, A, 3, 4, 5]]

In [12]:
def LinearTriangleAssemble(K,k,i,j,m):
    K[i,i] = K[i,i] + k[0,0]
    K[i,j] = K[i,j] + k[0,1]
    K[i,m] = K[i,m] + k[0,2]
    K[j,i] = K[j,i] + k[1,0]
    K[j,j] = K[j,j] + k[1,1]
    K[j,m] = K[j,m] + k[1,2]
    K[m,i] = K[m,i] + k[2,0]
    K[m,j] = K[m,j] + k[2,1]
    K[m,m] = K[m,m] + k[2,2]
    return K

In [13]:
def ElementForce(C,A):
    return C*A/3*Matrix([1,1,1])

In [14]:
def ForceAssemble(F,f,i,j,m):
    F[i]=F[i]+f[0]
    F[j]=F[j]+f[1]
    F[m]=F[m]+f[2]
    return F

In [15]:
def FEA_ф(inp, bc_node, bc_val, C):
    K=np.zeros(shape=(len(inp)+2, len(inp)+2))
    
    F=np.zeros(shape=(len(inp)+2,))
    
    max_coord=0
    for el in inp:
        bi, bj, bk, ci, cj, ck, A, i , j, m = el
        max_coord=max(max_coord,i,j,m)
        k=ElementStiffness( bi, bj, bk, ci, cj, ck, A)
        K=LinearTriangleAssemble(K, k, i, j, m)
        
        f=ElementForce(C,A)
        F=ForceAssemble(F,f,i,j,m)
        
        
    bc=np.array([bc_node, 
                bc_val]).T

    nd, nd=K.shape
    fdof=np.array([i for i in range(nd)]).T
    d=np.zeros(shape=(len(fdof),))
    Q=np.zeros(shape=(len(fdof),))


    pdof=bc[:,0].astype(int)
    dp=bc[:,1]
    fdof=np.delete(fdof, pdof, 0)
    s=np.linalg.solve(K[fdof,:][:,fdof],(F[fdof].T-np.dot(K[fdof,:][:,pdof],dp.T)).T)
    d[pdof]=dp
    d[fdof]=s.reshape(-1,)
#     Q=np.dot(K,d).T-F    
        
    return d

In [16]:
FEA_ф(inp, [2,4,5], [0,0,0], 2790)

array([217.96875 , 159.84375 ,   0.      , 123.515625,   0.      ,
         0.      ])

### 8.4

In [17]:
def solution(Xi, Yi, Xj, Yj, Xk, Yk, C = 2790):
    A=0.5*np.linalg.det(np.matrix(  
                [[1,Xi,Yi],
                [1,Xj,Yj],
                [1,Xk,Yk]]
                                 ))
    
    bi=Yj-Yk
    bj=Yk-Yi
    bk=Yi-Yj
    ci=Xk-Xj
    cj=Xi-Xk
    ck=Xj-Xi
  
    kd_1=1/(4*A)*np.matrix([[bi**2, bi*bj, bi*bk],
                            [bi*bj, bj**2, bj*bk],
                            [bi*bk, bj*bk, bk**2]])
    
    kd_2=1/(4*A)*np.matrix([[ci**2, ci*cj, ci*ck],
                            [ci*cj, cj**2, cj*ck],
                            [ci*ck, cj*ck, ck**2]])
    
    kd = kd_1 + kd_2    
    
    f=C*A/3*np.array([1,1,1])
    
    d=np.linalg.lstsq(kd, f, rcond=None)[0]
    
    div=1/(2*A)*np.dot(np.matrix([[bi, bj, bk],
                      [ci, cj, ck],]), d)
    
    return div, A/3*sum(d)

In [18]:
Xi=0.13
Yi=0.01
Xj=0.25
Yj=0.06
Xk=0.13
Yk=0.13

In [19]:
div, T = solution(Xi, Yi, Xj, Yj, Xk, Yk)

In [20]:
div

matrix([[-1.58502894e-14,  2.59096766e-14]])

In [21]:
T

-9.466330862652143e-34

In [22]:
# TODO - the answer is not the same as in the book

### 8.6

In [23]:
Xi=0.13
Yi=0.13
Xj=0.25
Yj=0.13
Xk=0.19
Yk=0.19

In [24]:
div, T = solution(Xi, Yi, Xj, Yj, Xk, Yk)

In [25]:
div

matrix([[-2.57283461e-15,  1.63220625e-14]])

In [26]:
T

-2.366582715663035e-34

In [27]:
# TODO - the answer is not the same as in the book

### 8.10

In [28]:
def solution(Xi, Yi, Xj, Ym, C = 2790):
    a=(Ym-Yi)/2
    b=(Xj-Xi)/2
    A=4*a*b
  
    kd_1=a/(6*b)*np.matrix([[2,-2,-1,1],
                          [-2,2,1,-1],
                          [-1,1,2,-2],
                          [1,-1,-2,2]])
    
    kd_2=b/(6*a)*np.matrix([[2,1,-1,-2],
                           [1,2,-2,-1],
                           [-1,-2,2,1],
                           [-2,-1,1,2]])
    
    kd=kd_1+kd_2
    
    f=C*A/4*np.array([1,1,1,1])

    
    d=np.linalg.lstsq(kd, f, rcond=None)[0]
    t=a
    s=b
    div=1/(2*A)*np.dot(np.matrix([[-(2*a-t),(2*a-t),t,-t],
                      [-(2*b-s),-s,s,(2*b-s)],]),d)
    
    return div, A/4*sum(d)

In [29]:
Xi=0.25
Yi=0.18
Xj=0.31
Ym=0.25

In [30]:
div, T = solution(Xi, Yi, Xj, Ym)

In [31]:
div

matrix([[ 2.84693885e-15, -1.16606724e-14]])

In [32]:
T

4.141519752410313e-34

In [33]:
# TODO - the answer is not the same as in the book

### 8.12

In [34]:
Xi=0.31
Yi=0.13
Xj=0.38
Ym=0.19

In [35]:
div, T = solution(Xi, Yi, Xj, Ym)

In [36]:
div

matrix([[-4.84174451e-15,  4.15643334e-16]])

In [37]:
T

2.0707598762051563e-34

In [38]:
# TODO - the answer is not the same as in the book